# Preprocessing

In [ ]:
import pickle
import json
import os
from preprocessing import clean_gt, clean_raw, label_entity

In [ ]:
import pickle
import json
import os
from preprocessing import clean_gt, clean_raw, label_entity
split = pickle.load(open('data/train_test_eval_filenames_new.pkl', 'rb'))
split

In [ ]:
#train = 94, test = 30, eval = 10

In [ ]:
# import random
# new_split = {"train":[], "test": [], "eval": []}
# for key in split:
#     for page in split[key]:
#         mag = page.split("_")[0].split("-")[0]
#         year = page.split("_")[1]
        
#         if mag == "dkm" and (year == "1941" or year == "2010"):
#             new_split["train"].append(page)
#         if (year == "1990"):
#             new_split["test"].append(page)
#         if mag =="sbz" and (year == "1895" or year == "1940" or year == "1965" or year == "2010"):
#             new_split["train"].append(page)

# eval_set = random.sample(new_split["train"], int(len(new_split["train"])/10)) #set 10% of train aside for eval
# for page in eval_set:
#     new_split["train"].remove(page)
# new_split["eval"] = eval_set

# with open('train_test_eval_filenames_new.pkl', 'wb') as out:
#     pickle.dump(new_split, out)

In [ ]:
"""
data = {
    "train": [],
    "test": [],
    "eval": []
}
gt_data = []
for mag in ["dkm", "sbz"]:
    for year in os.listdir(f'data/raw/link/{mag}'):
        with open(os.path.join("data/raw/link", mag, year)) as f:
            input_linked = json.load(f)
        with open(os.path.join("data/ground-truth", mag, year)) as f:
            gt = json.load(f)
        gt = clean_gt(gt)
        gt_data += gt
        input_linked = clean_raw(input_linked)

        #due to non-determinism in the flair NER:
        all_refs_gt = [g["page"]+g["coord"] for g in gt] 
        all_refs_linked = [ent["page"]+ent["coord"] for l in input_linked for ent in l]
        all_valid_refs = set(all_refs_gt).intersection(set(all_refs_linked))

        for ent_variations in input_linked:
            for key in split:
                ent_instances = []
                for ent in ent_variations:
                    if ent["page"] in split[key]:
                        if (ent["page"]+ent["coord"]) in all_valid_refs:
                            ent_instances.append({"ent": ent, "label": label_entity(ent, gt)})
                if ent_instances:
                    data[key].append(ent_instances)
"""

In [ ]:
"""
with open("data/processed.pkl", "wb") as f:
    pickle.dump(data, f)
"""

# Candidate Generation

### To get the relevant fastttext model uncomment and run the following cell

In [ ]:
# from gensim.models.fasttext import FastText, load_facebook_vectors
# model = load_facebook_vectors("cc.de.300.bin/cc.de.300.bin")
# model.save("./fasttext")

In [ ]:
import pickle
from tqdm.notebook import  tqdm
from feature_generation import candidates_to_features, process_fuseki_candidates

In [ ]:
import pickle
from tqdm.notebook import  tqdm
from feature_generation import candidates_to_features, process_fuseki_candidates
#AF:
from tqdm import tqdm
# load everytime you run this as we pop keys to keep data clean..
with open("data/processed.pkl", "rb") as f:
    data = pickle.load(f)

problematic_entities = []
for split in ["train", "eval", "test"]:
    ent_cand_label = []
    i = 0
    for entity_list in tqdm(data[split], smoothing=0.01):
        i += 1
        # Create candidates only for the first entry in the list as all the entity information is always the same
        # The only thing that changes are pages and page_coordinates
        
        # fuseki:
        unique_candidate_dict = get_candidates_fuseki(entity_list[0]["ent"])
        candidates = process_fuseki_candidates(unique_candidate_dict)
        #print(process_fuseki_candidates(unique_candidate_dict))
        #list_of_tuples.append((ent, processed_fuseki_cands, ent_dict["label"]))
        
        #metagrid
        ## candidates = create_metagrid_candidates(ent=entity_list[0]["ent"])
        # Generate the list of page_coordinates and the corresponding labels!
        coord_list = []
        gt_label = []
        for ent_dict in entity_list:
            ent = ent_dict["ent"]
            coord_list.append({
                "page": ent.pop("page", ""), 
                "coords": ent.pop("coord", "")
            })
            gt_label.append(ent_dict["label"])

        #if len(gt_label)!=1:
        #    problematic_entities.append({"ent_list": entity_list, "gt_labels": gt_label, "mag": coord_list})
        #gt_label = gt_label.pop()
        
        ent_cand_label.append({"entity": ent, "candidates": candidates, "occurences": coord_list, "gt_label": gt_label})
        if i % 100 == 0:
            with open(f"data/candidates/fuseki/candidates-gnd-{split}-{i}.pkl", "wb") as f:
                pickle.dump(ent_cand_label, f)
                    
    with open(f"data/candidates/fuseki/candidates-gnd-{split}.pkl", "wb") as f:
        pickle.dump(ent_cand_label, f)
print(problematic_entities)


# Feature Generation

In [ ]:
import pickle
from tqdm import  tqdm
from feature_generation import candidates_to_features

In [ ]:
generator = "fuseki" # or "metagrid"

In [ ]:
import pickle
from tqdm import  tqdm
from feature_generation import candidates_to_features
import pickle
from tqdm.notebook import  tqdm
from candidate_generation import create_metagrid_candidates, get_candidates_fuseki
generator = "fuseki" # or "metagrid"
for split in ["train", "eval", "test"]:
    with open(f"data/candidates/{generator}/candidates-gnd-{split}.pkl", "rb") as f:
        ent_cand_label = pickle.load(f)

    list_of_good_entities = []
    list_of_problematic_entities = []
    for ent_dict in tqdm(ent_cand_label):
        if len(set(ent_dict["gt_label"])) > 1:
            for label in set(ent_dict["gt_label"]):
                ent_dict["label"] = label
                features = candidates_to_features(ent=ent_dict["entity"], candidates=ent_dict["candidates"], gt_label=ent_dict["label"])
                ent_dict.update(features)
                list_of_problematic_entities.append(ent_dict.copy())
        else:
            ent_dict["label"] = set(ent_dict["gt_label"]).pop()
            features = candidates_to_features(ent=ent_dict["entity"], candidates=ent_dict["candidates"], gt_label=ent_dict["label"])
            ent_dict.update(features)
            list_of_good_entities.append(ent_dict)
            
    with open(f"data/features/{generator}/{split}.pkl", "wb") as f:
        pickle.dump(list_of_good_entities, file=f)
    
    with open(f"data/features/{generator}/{split}_problematic.pkl", "wb") as f:
        pickle.dump(list_of_problematic_entities, file=f)

# Unsupervised Features

The data is here:
1. https://github.com/felanders/ETH-CS4NLP-22-Project-Linking-GND/tree/fatih/data/input/raw
2. https://github.com/felanders/ETH-CS4NLP-22-Project-Linking-GND/tree/fatih/data/website_content_cache

In [ ]:
# import numpy as np
import logging
import pickle
from tqdm.notebook import  tqdm
import importlib
import unsupervised.raw_text_driver
importlib.reload(unsupervised.raw_text_driver)
import unsupervised.portal_dnb_driver
importlib.reload(unsupervised.portal_dnb_driver)
import unsupervised.data_loader
with open(f"data/features/train.pkl", "rb") as f:
    list_of_good_entities = pickle.load(f)

importlib.reload(unsupervised.data_loader)

# REPLACE THIS RAW DATA PATH
data_loader = unsupervised.data_loader.DataLoader(raw_data_path='/home/aheser/ETH-CS4NLP-22-Project-Linking-GND/data/input/raw')


In [14]:
generator = "fuseki" # or "metagrid"

In [ ]:
logging.basicConfig(level=logging.WARNING)
avg_distance_counter = 0
avg_distance = np.array((1,))
for split in ["train", "eval", "test"]:
# for split in ["eval"]:
    print('processing', split)
    with open(f"data/features/{generator}/{split}.pkl", "rb") as f:
        list_of_good_entities = pickle.load(f)
    
    some_counter = 0
    for current_entity in tqdm(list_of_good_entities):
        print(some_counter)
        some_counter = some_counter + 1
        distances = data_loader.get_context_distances(current_entity, similarity_measure='cosine_similarity', window_size=10)
        for feature_counter in range(len(current_entity['features'])):
            current_entity['features'][feature_counter].extend(distances[feature_counter])
#             print('avg distance', avg_distance, 'distance vec', distances[feature_counter])
            avg_distance = avg_distance + np.array(distances[feature_counter])
            avg_distance_counter = avg_distance_counter + 1
        
    
    with open(f"data/features/{generator}/unsupervised_{split}.pkl", "wb") as f:
        pickle.dump(list_of_good_entities, file=f)

avg_distance = avg_distance / avg_distance_counter
# problematic ones (we don't have any vectors, use mean)
for split in ["train", "eval", "test"]:
    print('processing', split)
    with open(f"data/features/{generator}/{split}_problematic.pkl", "rb") as f:
        list_of_good_entities = pickle.load(f)
    
    for current_entity in tqdm(list_of_good_entities):
        for feature_counter in range(len(current_entity['features'])):
            current_entity['features'][feature_counter].extend(avg_distance)
    
    with open(f"data/features/{generator}/unsupervised_{split}_problematic.pkl", "wb") as f:
        pickle.dump(list_of_good_entities, file=f)

# Ranking

## Load Data

In [ ]:
import pickle
from tqdm import tqdm
import numpy as np
from evaluation import perform_experiment, crossvalidate_experiment

In [ ]:
generator = "fuseki" # or "fuseki"

In [ ]:
d = {"train": {}, "eval": {}, "test": {}}
for split in ["train", "eval", "test"]:
    with open(f"data/features/{generator}/unsupervised_{split}.pkl", "rb") as f:
        d[split] = pickle.load(file=f)

# problematic entities
d_problem = {"train": {}, "eval": {}, "test": {}}
for split in ["train", "eval", "test"]:
    with open(f"data/features/{generator}/unsupervised_{split}_problematic.pkl", "rb") as f:
        d_problem[split] = pickle.load(file=f)

d_combined = {"train": d["train"] + d_problem["train"], "eval": d["eval"] + d_problem["eval"], "test": d["test"] + d_problem["test"]}

**Best scores we could get**

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
ent_scores, ment_scores = perform_experiment(
    keep_empty=True,
    do_sample=True,
    oversampling=3, # Multiple of how often we oversample y = 1
    balance=3, # multiple of y = 0 samples vs y = 1 samples
    train=d["train"] + d["eval"],
    eval=d["test"],
    model=ExtraTreesRegressor(n_estimators=100, random_state=0, criterion="squared_error", bootstrap=True),
    n_s=[1,10], # How many candidates do we keep
    thresholds=[0.01, 0.2], # Where do we cut off
    verbose=False # Print stuff
)

In [ ]:
print("\nEntity Level")
for score in ent_scores:
    print("N:", score["top_n"], "Threshold:", score["threshold"])
    score["score"].print_scores()

print("Mention Level")
for score in ment_scores:
    print("N:", score["top_n"], "Threshold:", score["threshold"])
    score["score"].print_scores()

Including problematic entities

In [ ]:
ent_scores, ment_scores = perform_experiment(
    keep_empty=True,
    do_sample=True,
    oversampling=3, # Multiple of how often we oversample y = 1
    balance=3, # multiple of y = 0 samples vs y = 1 samples
    train=d_combined["train"] + d_combined["eval"],
    eval=d_combined["test"],
    model=ExtraTreesRegressor(n_estimators=100, random_state=0, criterion="squared_error", bootstrap=True),
    n_s=[1,10], # How many candidates do we keep
    thresholds=[0.01, 0.2], # Where do we cut off
    verbose=False # Print stuff
)

In [ ]:
print("\nEntity Level")
for score in ent_scores:
    print("N:", score["top_n"], "Threshold:", score["threshold"])
    score["score"].print_scores()

print("Mention Level")
for score in ment_scores:
    print("N:", score["top_n"], "Threshold:", score["threshold"])
    score["score"].print_scores()